In [1]:
# Jupyter Notebook Code
from code_gen_result_analysis import CodeAnalysis, cal_err_bar, bootstrap_resampling  # Assuming code_analysis.py is the name of the file
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Function to style the DataFrame
def highlight_cols(df):
     # We will create a custom style for the headers and vertical lines
     styles = [
               dict(selector="th.col_heading",
                    props=[("text-align", "center"),
                              ("border-right", "2px solid #6c6c6c"),
                              ("border-left", "2px solid #6c6c6c")]),
               dict(selector="th.col_heading.level0",
                    props=[("border-top", "2px solid #6c6c6c")]),
               dict(selector="th",
                    props=[("font-size", "12px")]),
               dict(selector="td",
                    props=[("text-align", "center")]),
               # Customize the boundary for your specific DataFrame structure
               # Adjust "4" and "10" according to your DataFrame's column indices
               dict(selector=f"th:nth-child(5), td:nth-child(5)",
                    props=[("border-left", "2px solid #6c6c6c")]),
               dict(selector=f"th:nth-child(11), td:nth-child(11)",
                    props=[("border-left", "2px solid #6c6c6c")])
          ]
     return df.style.set_table_styles(styles).set_properties(**{'width': '120px', 'text-align': 'center'}).hide_index()

label_mapping = {
     'If Condition': 'If Condition', #If Condition
     'Elif Condition': 'If Condition',
     'If Body': 'If Body',
     'Elif Body': 'If Body',
     'Else Reasoning': 'If Body',
     'Loop Body': 'Loop Body',
     'Define Stop Criteria': 'Loop Body',
     'List Comprehension': 'List Comprehension',
     'Lambda Expressions': 'List Comprehension',
     'Generator Expressions': 'List Comprehension',
}
def get_summary_df(directory_path, label_mapping, weighted=False, use_max_range=True,gen_code_pass_col = 'post_process_pass_ratio'):
     analysis = CodeAnalysis(weighted=weighted, use_max_range=use_max_range, gen_code_pass_col=gen_code_pass_col)  # Pass additional parameters if different from defaults
     # Analyze results and structure the summary DataFrame
     summary_df = analysis.analyze_results_in_folder(directory_path, label_mapping=label_mapping)
     # Sort the DataFrame by 'Model Name'
     summary_df_sorted = summary_df.sort_values(by='Model Name')
     # Creating MultiIndex for columns to include subheaders
     reasons_columns = [('Reason Categories Pass Ratio', col.replace('_', ' ')) for col in analysis.labels_reasons]
     horizons_columns = [('Horizon Categories Pass Ratio', col.replace('_', ' ')) for col in analysis.labels_horizons]
     columns = [('General', 'Model Name'), ('General', 'Generation Mode'), ('General', 'Code Task'), ('General', 'All Pass Ratio')] + reasons_columns + horizons_columns
     summary_df_sorted.columns = pd.MultiIndex.from_tuples(columns)

     # Apply the styling function to your summary DataFrame
     styled_df = highlight_cols(summary_df_sorted)
     return summary_df_sorted, styled_df

# Function to split the ratio column into 'Passed Count' and 'Total Count'
def split_ratio_for_general(df, debug_print=False):
     if debug_print:
          print(df[('General', 'All Pass Ratio')])  # Print before split
     df[('General', 'Passed Count')] = df[('General', 'All Pass Ratio')].str.extract('\((\d+)/').squeeze().astype(int)
     df[('General', 'Total Count')] = df[('General', 'All Pass Ratio')].str.extract('/(\d+)\)').squeeze().astype(int)
     if debug_print:
          print(df[('General', 'Passed Count')])
          print(df[('General', 'Total Count')])
     return df

# Function to analyze the DataFrame by 'Model Name' and 'Generation Mode'
def analyze_by_model_and_mode_for_general(df, report_err_bar=True):
     df = split_ratio_for_general(df)
     grouped_df = df.groupby([('General', 'Model Name'), ('General', 'Generation Mode')]).sum()

     total_counts = grouped_df[('General', 'Total Count')]
     pass_counts = grouped_df[('General', 'Passed Count')]

     percentages = pass_counts / total_counts * 100

     if report_err_bar:
          percentages, err_bar = cal_err_bar(pass_counts, total_counts)
          err_bar = pd.Series(err_bar, index=grouped_df.index)
          grouped_df[('General', 'All Pass Ratio')] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
     else:
          grouped_df[('General', 'All Pass Ratio')] = percentages.map("{:.3f}%".format)

     return grouped_df

# Function to split the ratio column into 'Passed Count' and 'Total Count'
# for Reason Categories Pass Ratio and Horizon Categories Pass Ratio
def split_ratio(df):
     reason_categories = ['List Comprehension', 'If Condition','If Body', 'Stream Operations', 'Loop Body', 'Super Call']
     horizon_categories = ['Short-Range', 'Medium-Range', 'Long-Range', 'Variable', 'Global Variable', 'Function', 'Class', 'Library', 'Interface']

     for category in reason_categories:
          df[('Reason Categories Pass Ratio', category + ' Passed Count')] = df[('Reason Categories Pass Ratio', category)].str.extract('\((\d+)/').squeeze().fillna(0).astype(int)
          df[('Reason Categories Pass Ratio', category + ' Total Count')] = df[('Reason Categories Pass Ratio', category)].str.extract('/(\d+)\)').squeeze().fillna(0).astype(int)

     for category in horizon_categories:
          df[('Horizon Categories Pass Ratio', category + ' Passed Count')] = df[('Horizon Categories Pass Ratio', category)].str.extract('\((\d+)/').squeeze().fillna(0).astype(int)
          df[('Horizon Categories Pass Ratio', category + ' Total Count')] = df[('Horizon Categories Pass Ratio', category)].str.extract('/(\d+)\)').squeeze().fillna(0).astype(int)

     return df

def analyze_by_model_and_mode(df, report_err_bar=True):
     df = split_ratio(df)
     grouped_df = df.groupby([('General', 'Model Name'), ('General', 'Generation Mode')]).sum()

     horizon_categories = ['Short-Range', 'Medium-Range', 'Long-Range', 'Variable', 'Global Variable', 'Function', 'Class', 'Library', 'Interface']
     reason_categories = ['List Comprehension', 'If Condition','If Body', 'Stream Operations', 'Loop Body', 'Super Call']

     if report_err_bar:
          for category in horizon_categories:
               total_counts = grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')]
               pass_counts = grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')]
               percentages = pass_counts / total_counts * 100

               if report_err_bar:
                    percentages, err_bar = cal_err_bar(pass_counts, total_counts)
                    err_bar = pd.Series(err_bar, index=grouped_df.index)
                    grouped_df[('Horizon Categories Pass Ratio', category)] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
               else:
                    grouped_df[('Horizon Categories Pass Ratio', category)] = percentages.map("{:.3f}%".format)

               grouped_df.drop([('Horizon Categories Pass Ratio', category + ' Passed Count'), ('Horizon Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

          for category in reason_categories:
               total_counts = grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')]
               pass_counts = grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')]
               percentages = pass_counts / total_counts * 100

               if report_err_bar:
                    percentages, err_bar = cal_err_bar(pass_counts, total_counts)
                    err_bar = pd.Series(err_bar, index=grouped_df.index)
                    grouped_df[('Reason Categories Pass Ratio', category)] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
               else:
                    grouped_df[('Reason Categories Pass Ratio', category)] = percentages.map("{:.3f}%".format)

               grouped_df.drop([('Reason Categories Pass Ratio', category + ' Passed Count'), ('Reason Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)
     else:
          for category in horizon_categories:
               pass_ratio = grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')] / grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')] * 100
               grouped_df[('Horizon Categories Pass Ratio', category)] = pass_ratio.map("{:.3f}%".format) + '(' + grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')].astype(str) + '/' + grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')].astype(str) + ')'
               grouped_df.drop([('Horizon Categories Pass Ratio', category + ' Passed Count'), ('Horizon Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

          for category in reason_categories:
               pass_ratio = grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')] / grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')] * 100
               grouped_df[('Reason Categories Pass Ratio', category)] = pass_ratio.map("{:.3f}%".format) + '(' + grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')].astype(str) + '/' + grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')].astype(str) + ')'
               grouped_df.drop([('Reason Categories Pass Ratio', category + ' Passed Count'), ('Reason Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

     return grouped_df

def clean_df(df, drop_general=False, drop_range=True, drop_nan=True, drop_zero=True):
     if drop_general:
          df = df.drop(columns=[col for col in df.columns if 'Passed Count' in col or 'Total Count' in col], errors='ignore')

     if drop_range:
          df = df.drop(columns=[('Horizon Categories Pass Ratio', 'Short-Range'), ('Horizon Categories Pass Ratio', 'Medium-Range'), ('Horizon Categories Pass Ratio', 'Long-Range')], errors='ignore')

     if drop_nan:
          df = df.loc[:, ~(df == 'nan%(0/0)').all()]

     if drop_zero:
          df = df.loc[:, ~(df == '0.0 ± 0.0').all()]

     return df

# def analyze_and_save(directory_path, label_mapping, save_file_name, gen_code_pass_col='post_process_pass_ratio'):
def analyze_and_save(directory_path, label_mapping, save_file_name, gen_code_pass_col='gen_code_pass_ratio'):
     summary_df_sorted, styled_df = get_summary_df(directory_path, label_mapping, gen_code_pass_col = gen_code_pass_col)
     general_analyzed_df = analyze_by_model_and_mode_for_general(summary_df_sorted)
     display(general_analyzed_df)
     analyzed_df = analyze_by_model_and_mode(summary_df_sorted)
     cleaned_df = clean_df(analyzed_df)
     display(cleaned_df)
     cleaned_df.to_csv(save_file_name)

# Set display options for pandas DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [2]:
def analyze_and_display_results_for_settings():
    settings = [
        {"language": "Python", "type": "Completion", "directory": "../Analysis_Results/storage_server/Python_all_res/Completion/4th_post_process_reason_update/Update_labels", "save_file": "Python_Completion_grouped.csv"},
        {"language": "Python", "type": "Infilling", "directory": "../Analysis_Results/storage_server/Python_all_res/Infilling/4th_post_process_reason_update/Update_labels", "save_file": "Python_Infilling_grouped.csv"},
        {"language": "Java", "type": "Completion", "directory": "../Analysis_Results/storage_server/Java_all_res/Completion/4th_post_process_reason_update/Update_labels", "save_file": "Java_Completion_grouped.csv"},
        {"language": "Java", "type": "Infilling", "directory": "../Analysis_Results/storage_server/Java_all_res/Infilling/4th_post_process_reason_update/Update_labels", "save_file": "Java_Infilling_grouped.csv"}
    ]

    for setting in settings:
        print(f"Analyzing {setting['language']} {setting['type']}")
        analyze_and_save(setting['directory'], label_mapping, setting['save_file'])
        print(f"Analysis and saving completed for {setting['language']} {setting['type']}\n\n")

# Example usage
analyze_and_display_results_for_settings()

Analyzing Python Completion


General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    no_afterlines                        39   
Meta-Llama-3-8B-Instruct     no_afterlines                        14   
claude-3-haiku-20240307      no_afterlines                        14   
claude-3-opus-20240229       no_afterlines                         8   
claude-3-sonnet-20240229     no_afterlines                         7   
deepseek-coder-1.3b-instruct no_afterlines                         5   
deepseek-coder-7b-instruct   no_afterlines                        46   
gpt-3.5-turbo-0125           no_afterlines                        68   
gpt-4-turbo                  no_afterlines                        96   
phi-3-mini-4k                no_afterlines                         2   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    no_afterlines                      212   
Meta-Llama-3-8B-Instruct     no_afterlines                      212   
claude-3-haiku-20240307      no_afterlines                      212   
claude-3-opus-20240229       no_afterlines                      212   
claude-3-sonnet-20240229     no_afterlines                      212   
deepseek-coder-1.3b-instruct no_afterlines                      212   
deepseek-coder-7b-instruct   no_afterlines                      212   
gpt-3.5-turbo-0125           no_afterlines                      212   
gpt-4-turbo                  no_afterlines                      212   
phi-3-mini-4k                no_afterlines                      212   

                                                                        
                                                        All Pass Ratio  
(General, Model Name)        (General, Generation Mode)                 
Meta-Llama-3-70B-Instruct    no_afterlines                  18.4 ± 5.3  
Meta-Llama-3-8B-Instruct     no_afterlines                   6.6 ± 3.3  
claude-3-haiku-20240307      no_afterlines                   6.6 ± 3.3  
claude-3-opus-20240229       no_afterlines                   3.8 ± 2.6  
claude-3-sonnet-20240229     no_afterlines                   3.3 ± 2.4  
deepseek-coder-1.3b-instruct no_afterlines                   2.4 ± 2.0  
deepseek-coder-7b-instruct   no_afterlines                  21.7 ± 5.5  
gpt-3.5-turbo-0125           no_afterlines                  32.0 ± 6.3  
gpt-4-turbo                  no_afterlines                  45.2 ± 6.6  
phi-3-mini-4k                no_afterlines                   0.9 ± 1.3

General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    no_afterlines                        39   
Meta-Llama-3-8B-Instruct     no_afterlines                        14   
claude-3-haiku-20240307      no_afterlines                        14   
claude-3-opus-20240229       no_afterlines                         8   
claude-3-sonnet-20240229     no_afterlines                         7   
deepseek-coder-1.3b-instruct no_afterlines                         5   
deepseek-coder-7b-instruct   no_afterlines                        46   
gpt-3.5-turbo-0125           no_afterlines                        68   
gpt-4-turbo                  no_afterlines                        96   
phi-3-mini-4k                no_afterlines                         2   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    no_afterlines                      212   
Meta-Llama-3-8B-Instruct     no_afterlines                      212   
claude-3-haiku-20240307      no_afterlines                      212   
claude-3-opus-20240229       no_afterlines                      212   
claude-3-sonnet-20240229     no_afterlines                      212   
deepseek-coder-1.3b-instruct no_afterlines                      212   
deepseek-coder-7b-instruct   no_afterlines                      212   
gpt-3.5-turbo-0125           no_afterlines                      212   
gpt-4-turbo                  no_afterlines                      212   
phi-3-mini-4k                no_afterlines                      212   

                                                        Horizon Categories Pass Ratio  \
                                                                             Variable   
(General, Model Name)        (General, Generation Mode)                                 
Meta-Llama-3-70B-Instruct    no_afterlines                                 18.0 ± 5.2   
Meta-Llama-3-8B-Instruct     no_afterlines                                  6.8 ± 3.4   
claude-3-haiku-20240307      no_afterlines                                  6.8 ± 3.4   
claude-3-opus-20240229       no_afterlines                                  3.9 ± 2.6   
claude-3-sonnet-20240229     no_afterlines                                  3.4 ± 2.4   
deepseek-coder-1.3b-instruct no_afterlines                                  2.4 ± 2.1   
deepseek-coder-7b-instruct   no_afterlines                                 22.4 ± 5.7   
gpt-3.5-turbo-0125           no_afterlines                                 32.5 ± 6.4   
gpt-4-turbo                  no_afterlines                                 45.6 ± 6.8   
phi-3-mini-4k                no_afterlines                                  1.0 ± 1.3   

                                                                      \
                                                            Function   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    no_afterlines               11.0 ± 20.5   
Meta-Llama-3-8B-Instruct     no_afterlines                 0.0 ± 0.0   
claude-3-haiku-20240307      no_afterlines                 0.0 ± 0.0   
claude-3-opus-20240229       no_afterlines                 0.0 ± 0.0   
claude-3-sonnet-20240229     no_afterlines                 0.0 ± 0.0   
deepseek-coder-1.3b-instruct no_afterlines                 0.0 ± 0.0   
deepseek-coder-7b-instruct   no_afterlines                 0.0 ± 0.0   
gpt-3.5-turbo-0125           no_afterlines               11.1 ± 20.5   
gpt-4-turbo                  no_afterlines               22.0 ± 27.0   
phi-3-mini-4k                no_afterlines                 0.0 ± 0.0   

                                                                      \
                                 

Analysis and saving completed for Python Completion


Analyzing Python Infilling


General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    with_afterlines                     109   
Meta-Llama-3-8B-Instruct     with_afterlines                      23   
claude-3-haiku-20240307      with_afterlines                      43   
claude-3-opus-20240229       with_afterlines                     114   
claude-3-sonnet-20240229     with_afterlines                      94   
deepseek-coder-1.3b-instruct with_afterlines                       9   
deepseek-coder-7b-instruct   with_afterlines                      12   
gpt-3.5-turbo-0125           with_afterlines                      60   
gpt-4-turbo                  with_afterlines                     219   
phi-3-mini-4k                with_afterlines                       4   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    with_afterlines                    382   
Meta-Llama-3-8B-Instruct     with_afterlines                    382   
claude-3-haiku-20240307      with_afterlines                    382   
claude-3-opus-20240229       with_afterlines                    382   
claude-3-sonnet-20240229     with_afterlines                    382   
deepseek-coder-1.3b-instruct with_afterlines                    382   
deepseek-coder-7b-instruct   with_afterlines                    382   
gpt-3.5-turbo-0125           with_afterlines                    382   
gpt-4-turbo                  with_afterlines                    382   
phi-3-mini-4k                with_afterlines                    382   

                                                                        
                                                        All Pass Ratio  
(General, Model Name)        (General, Generation Mode)                 
Meta-Llama-3-70B-Instruct    with_afterlines                28.6 ± 4.5  
Meta-Llama-3-8B-Instruct     with_afterlines                 6.0 ± 2.4  
claude-3-haiku-20240307      with_afterlines                11.2 ± 3.2  
claude-3-opus-20240229       with_afterlines                29.8 ± 4.6  
claude-3-sonnet-20240229     with_afterlines                24.6 ± 4.3  
deepseek-coder-1.3b-instruct with_afterlines                 2.3 ± 1.5  
deepseek-coder-7b-instruct   with_afterlines                 3.1 ± 1.7  
gpt-3.5-turbo-0125           with_afterlines                15.7 ± 3.6  
gpt-4-turbo                  with_afterlines                57.3 ± 4.9  
phi-3-mini-4k                with_afterlines                 1.0 ± 1.0

...

General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    no_afterlines                        36   
Meta-Llama-3-8B-Instruct     no_afterlines                         6   
claude-3-haiku-20240307      no_afterlines                        18   
claude-3-opus-20240229       no_afterlines                        79   
claude-3-sonnet-20240229     no_afterlines                        32   
deepseek-coder-1.3b-instruct no_afterlines                         3   
deepseek-coder-7b-instruct   no_afterlines                         7   
gpt-3.5-turbo-0125           no_afterlines                        10   
gpt-4-turbo                  no_afterlines                        27   
phi-3-mini-4k                no_afterlines                         1   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    no_afterlines                      286   
Meta-Llama-3-8B-Instruct     no_afterlines                      286   
claude-3-haiku-20240307      no_afterlines                      286   
claude-3-opus-20240229       no_afterlines                      286   
claude-3-sonnet-20240229     no_afterlines                      286   
deepseek-coder-1.3b-instruct no_afterlines                      286   
deepseek-coder-7b-instruct   no_afterlines                      286   
gpt-3.5-turbo-0125           no_afterlines                      286   
gpt-4-turbo                  no_afterlines                      286   
phi-3-mini-4k                no_afterlines                      286   

                                                        Horizon Categories Pass Ratio  \
                                                                             Variable   
(General, Model Name)        (General, Generation Mode)                                 
Meta-Llama-3-70B-Instruct    no_afterlines                                 10.6 ± 4.1   
Meta-Llama-3-8B-Instruct     no_afterlines                                  1.8 ± 1.8   
claude-3-haiku-20240307      no_afterlines                                  4.1 ± 2.6   
claude-3-opus-20240229       no_afterlines                                 25.2 ± 5.8   
claude-3-sonnet-20240229     no_afterlines                                  9.6 ± 3.9   
deepseek-coder-1.3b-instruct no_afterlines                                  1.4 ± 1.5   
deepseek-coder-7b-instruct   no_afterlines                                  1.8 ± 1.8   
gpt-3.5-turbo-0125           no_afterlines                                  3.7 ± 2.5   
gpt-4-turbo                  no_afterlines                                 10.6 ± 4.2   
phi-3-mini-4k                no_afterlines                                  0.5 ± 0.9   

                                                                         \
                                                        Global Variable   
(General, Model Name)        (General, Generation Mode)                   
Meta-Llama-3-70B-Instruct    no_afterlines                   12.0 ± 4.6   
Meta-Llama-3-8B-Instruct     no_afterlines                    2.2 ± 2.1   
claude-3-haiku-20240307      no_afterlines                    7.1 ± 3.7   
claude-3-opus-20240229       no_afterlines                   30.5 ± 6.6   
claude-3-sonnet-20240229     no_afterlines                   10.9 ± 4.5   
deepseek-coder-1.3b-instruct no_afterlines                    0.0 ± 0.0   
deepseek-coder-7b-instruct   no_afterlines                    3.3 ± 2.6   
gpt-3.5-turbo-0125           no_afterlines                    2.2 ± 2.1   
gpt-4-turbo                  no_afterlines                    8.7 ± 4.0   
phi-3-mini-4k                no_afterlines                    0.0 ± 0.0   

                                                                  

Analysis and saving completed for Java Completion


Analyzing Java Infilling


General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    with_afterlines                     119   
Meta-Llama-3-8B-Instruct     with_afterlines                      21   
claude-3-haiku-20240307      with_afterlines                      19   
claude-3-opus-20240229       with_afterlines                     159   
claude-3-sonnet-20240229     with_afterlines                     103   
deepseek-coder-1.3b-instruct with_afterlines                       4   
deepseek-coder-7b-instruct   with_afterlines                       3   
gpt-3.5-turbo-0125           with_afterlines                      38   
gpt-4-turbo                  with_afterlines                     181   
phi-3-mini-4k                with_afterlines                      11   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    with_afterlines                    283   
Meta-Llama-3-8B-Instruct     with_afterlines                    283   
claude-3-haiku-20240307      with_afterlines                    283   
claude-3-opus-20240229       with_afterlines                    283   
claude-3-sonnet-20240229     with_afterlines                    283   
deepseek-coder-1.3b-instruct with_afterlines                    283   
deepseek-coder-7b-instruct   with_afterlines                    283   
gpt-3.5-turbo-0125           with_afterlines                    283   
gpt-4-turbo                  with_afterlines                    283   
phi-3-mini-4k                with_afterlines                    283   

                                                                        
                                                        All Pass Ratio  
(General, Model Name)        (General, Generation Mode)                 
Meta-Llama-3-70B-Instruct    with_afterlines                42.0 ± 5.8  
Meta-Llama-3-8B-Instruct     with_afterlines                 7.4 ± 3.1  
claude-3-haiku-20240307      with_afterlines                 6.7 ± 2.9  
claude-3-opus-20240229       with_afterlines                56.2 ± 5.8  
claude-3-sonnet-20240229     with_afterlines                36.4 ± 5.6  
deepseek-coder-1.3b-instruct with_afterlines                 1.4 ± 1.4  
deepseek-coder-7b-instruct   with_afterlines                 1.1 ± 1.2  
gpt-3.5-turbo-0125           with_afterlines                13.4 ± 3.9  
gpt-4-turbo                  with_afterlines                64.0 ± 5.6  
phi-3-mini-4k                with_afterlines                 3.9 ± 2.3

General  \
                                                        Passed Count   
(General, Model Name)        (General, Generation Mode)                
Meta-Llama-3-70B-Instruct    with_afterlines                     119   
Meta-Llama-3-8B-Instruct     with_afterlines                      21   
claude-3-haiku-20240307      with_afterlines                      19   
claude-3-opus-20240229       with_afterlines                     159   
claude-3-sonnet-20240229     with_afterlines                     103   
deepseek-coder-1.3b-instruct with_afterlines                       4   
deepseek-coder-7b-instruct   with_afterlines                       3   
gpt-3.5-turbo-0125           with_afterlines                      38   
gpt-4-turbo                  with_afterlines                     181   
phi-3-mini-4k                with_afterlines                      11   

                                                                     \
                                                        Total Count   
(General, Model Name)        (General, Generation Mode)               
Meta-Llama-3-70B-Instruct    with_afterlines                    283   
Meta-Llama-3-8B-Instruct     with_afterlines                    283   
claude-3-haiku-20240307      with_afterlines                    283   
claude-3-opus-20240229       with_afterlines                    283   
claude-3-sonnet-20240229     with_afterlines                    283   
deepseek-coder-1.3b-instruct with_afterlines                    283   
deepseek-coder-7b-instruct   with_afterlines                    283   
gpt-3.5-turbo-0125           with_afterlines                    283   
gpt-4-turbo                  with_afterlines                    283   
phi-3-mini-4k                with_afterlines                    283   

                                                        Horizon Categories Pass Ratio  \
                                                                             Variable   
(General, Model Name)        (General, Generation Mode)                                 
Meta-Llama-3-70B-Instruct    with_afterlines                               41.0 ± 6.4   
Meta-Llama-3-8B-Instruct     with_afterlines                                6.5 ± 3.2   
claude-3-haiku-20240307      with_afterlines                                6.5 ± 3.2   
claude-3-opus-20240229       with_afterlines                               56.0 ± 6.3   
claude-3-sonnet-20240229     with_afterlines                               34.0 ± 6.1   
deepseek-coder-1.3b-instruct with_afterlines                                1.7 ± 1.7   
deepseek-coder-7b-instruct   with_afterlines                                1.3 ± 1.5   
gpt-3.5-turbo-0125           with_afterlines                               12.1 ± 4.2   
gpt-4-turbo                  with_afterlines                               63.4 ± 6.2   
phi-3-mini-4k                with_afterlines                                4.3 ± 2.6   

                                                                         \
                                                        Global Variable   
(General, Model Name)        (General, Generation Mode)                   
Meta-Llama-3-70B-Instruct    with_afterlines                 34.4 ± 7.4   
Meta-Llama-3-8B-Instruct     with_afterlines                  5.7 ± 3.6   
claude-3-haiku-20240307      with_afterlines                  5.7 ± 3.6   
claude-3-opus-20240229       with_afterlines                 57.4 ± 7.8   
claude-3-sonnet-20240229     with_afterlines                 35.0 ± 7.5   
deepseek-coder-1.3b-instruct with_afterlines                  1.3 ± 1.8   
deepseek-coder-7b-instruct   with_afterlines                  0.6 ± 1.2   
gpt-3.5-turbo-0125           with_afterlines                 14.7 ± 5.4   
gpt-4-turbo                  with_afterlines                 51.7 ± 7.8   
phi-3-mini-4k                with_afterlines                  1.9 ± 2.1   

                                                                  

Analysis and saving completed for Java Infilling


